# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    def get_list_of_sentences(text):
        return text.split('\n')
    
    def tokenize_list_of_sentences(list_of_sentences):
        tokenized_list_of_sentences = [[word for word in sentence.split()] for sentence in list_of_sentences]
        return tokenized_list_of_sentences
    
    def transcribe_to_ints(tokenized_list_of_sentences, word_to_int):
        transcription = [[word_to_int[word] for word in sentence] for sentence in tokenized_list_of_sentences]
        return transcription
    
    def pipeline(text, mapping):
        list_of_sentences = get_list_of_sentences(text)
        tokenized_list_of_sentences = tokenize_list_of_sentences(list_of_sentences)
        transcription = transcribe_to_ints(tokenized_list_of_sentences, mapping)
        return transcription
    
    EOS_id = target_vocab_to_int['<EOS>']
    source_id_text = pipeline(source_text, source_vocab_to_int)
    target_id_text = [ids + [EOS_id] for ids in pipeline(target_text, target_vocab_to_int)]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    
    inputs = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None))
    learning_rate = tf.placeholder(tf.float32, shape=())
    keep_prob = tf.placeholder(tf.float32, shape=(), name='keep_prob')
    target_seq_len = tf.placeholder(tf.int32, shape=(None,), name='target_sequence_length')
    max_target_seq_len = tf.reduce_max(target_seq_len, name='max_target_len')
    source_seq_len = tf.placeholder(tf.int32, shape=(None,), name='source_sequence_length')
    
    return (inputs,
            targets,
            learning_rate,
            keep_prob,
            target_seq_len,
            max_target_seq_len,source_seq_len)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    output_slice = target_data[:,:-1]
    paddings = [[0, 0], [1, 0]]
    pad_value = target_vocab_to_int['<GO>']
    decoder_input = tf.pad(output_slice,
                           paddings=paddings,
                           mode='CONSTANT',
                           constant_values=pad_value)
    
    return decoder_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    word_vecs = tf.contrib.layers.embed_sequence(rnn_inputs,
                                                 vocab_size=source_vocab_size,
                                                 embed_dim=encoding_embedding_size,
                                                 initializer=tf.variance_scaling_initializer)
    
    def single_cell():
        lstm_layer = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
        drop = tf.nn.rnn_cell.DropoutWrapper(lstm_layer, output_keep_prob=keep_prob)
        return drop
    multi_lstm = tf.nn.rnn_cell.MultiRNNCell([single_cell() for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(multi_lstm, word_vecs, dtype=tf.float32)

    return outputs, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)

    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                              helper,
                                              encoder_state,
                                              output_layer=output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,
                                                      impute_finished=False,
                                                      maximum_iterations=max_summary_length)
    return outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    start_ids = tf.constant(start_of_sequence_id, shape=(batch_size,))
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_ids, end_of_sequence_id)
#     dec_cell_drop = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
    
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                              helper,
                                              encoder_state,
                                              output_layer=output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,
                                                      impute_finished=True,
                                                      maximum_iterations=max_target_sequence_length)
    
    
    return outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    from tensorflow.python.layers.core import Dense
        
    dec_embeddings = tf.Variable(tf.random_uniform((target_vocab_size, decoding_embedding_size)))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    cell = lambda: tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
    drop = lambda: tf.nn.rnn_cell.DropoutWrapper(cell(), output_keep_prob=keep_prob)
    dec_cell = tf.nn.rnn_cell.MultiRNNCell([drop() for _ in range(num_layers)])
    
    output_layer = Dense(target_vocab_size, kernel_initializer=tf.variance_scaling_initializer)

    with tf.variable_scope('decoder'):
        training_logits = decoding_layer_train(encoder_state,
                                               dec_cell,
                                               dec_embed_input,
                                               target_sequence_length,
                                               max_target_sequence_length,
                                               output_layer,
                                               keep_prob)

    with tf.variable_scope('decoder', reuse=True):
        inference_logits = decoding_layer_infer(encoder_state,
                                            dec_cell,
                                            dec_embeddings,
                                            target_vocab_to_int['<GO>'],
                                            target_vocab_to_int['<EOS>'],
                                            max_target_sequence_length,
                                            target_vocab_size,
                                            output_layer,
                                            batch_size,
                                            keep_prob)
    
    return training_logits, inference_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    enc_outputs, enc_state = encoding_layer(input_data,
                                                    rnn_size,
                                                    num_layers,
                                                    keep_prob,
                                                    source_sequence_length,
                                                    source_vocab_size,
                                                    enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    training_logits, inference_logits = decoding_layer(dec_input,
                                                       enc_state,
                                                       target_sequence_length,
                                                       max_target_sentence_length,
                                                       rnn_size,
                                                       num_layers,
                                                       target_vocab_to_int,
                                                       target_vocab_size,
                                                       batch_size,
                                                       keep_prob,
                                                       dec_embedding_size)
    
    return training_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 100
decoding_embedding_size = 100
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2329, Validation Accuracy: 0.3096, Loss: 5.2087
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 4.3889
Epoch   0 Batch    3/269 - Train Accuracy: 0.2594, Validation Accuracy: 0.3252, Loss: 4.4263
Epoch   0 Batch    4/269 - Train Accuracy: 0.2691, Validation Accuracy: 0.3420, Loss: 4.1146
Epoch   0 Batch    5/269 - Train Accuracy: 0.2701, Validation Accuracy: 0.3426, Loss: 3.9774
Epoch   0 Batch    6/269 - Train Accuracy: 0.3138, Validation Accuracy: 0.3427, Loss: 3.7585
Epoch   0 Batch    7/269 - Train Accuracy: 0.3179, Validation Accuracy: 0.3495, Loss: 3.6042
Epoch   0 Batch    8/269 - Train Accuracy: 0.2953, Validation Accuracy: 0.3586, Loss: 3.6020
Epoch   0 Batch    9/269 - Train Accuracy: 0.3257, Validation Accuracy: 0.3661, Loss: 3.4109
Epoch   0 Batch   10/269 - Train Accuracy: 0.3037, Validation Accuracy: 0.3719, Loss: 3.4539
Epoch   0 Batch   11/269 - Train Accuracy: 0.3381, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.3701, Validation Accuracy: 0.4450, Loss: 2.0867
Epoch   0 Batch   91/269 - Train Accuracy: 0.4298, Validation Accuracy: 0.4643, Loss: 1.9815
Epoch   0 Batch   92/269 - Train Accuracy: 0.4280, Validation Accuracy: 0.4527, Loss: 1.9652
Epoch   0 Batch   93/269 - Train Accuracy: 0.4321, Validation Accuracy: 0.4413, Loss: 1.8869
Epoch   0 Batch   94/269 - Train Accuracy: 0.4144, Validation Accuracy: 0.4435, Loss: 1.9595
Epoch   0 Batch   95/269 - Train Accuracy: 0.4387, Validation Accuracy: 0.4624, Loss: 1.9188
Epoch   0 Batch   96/269 - Train Accuracy: 0.4303, Validation Accuracy: 0.4618, Loss: 1.9079
Epoch   0 Batch   97/269 - Train Accuracy: 0.4311, Validation Accuracy: 0.4580, Loss: 1.9290
Epoch   0 Batch   98/269 - Train Accuracy: 0.4454, Validation Accuracy: 0.4537, Loss: 1.8728
Epoch   0 Batch   99/269 - Train Accuracy: 0.3927, Validation Accuracy: 0.4514, Loss: 2.0046
Epoch   0 Batch  100/269 - Train Accuracy: 0.4585, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.4346, Validation Accuracy: 0.4511, Loss: 1.2649
Epoch   0 Batch  180/269 - Train Accuracy: 0.4109, Validation Accuracy: 0.4396, Loss: 1.2437
Epoch   0 Batch  181/269 - Train Accuracy: 0.4062, Validation Accuracy: 0.4510, Loss: 1.2652
Epoch   0 Batch  182/269 - Train Accuracy: 0.4224, Validation Accuracy: 0.4534, Loss: 1.2663
Epoch   0 Batch  183/269 - Train Accuracy: 0.4962, Validation Accuracy: 0.4561, Loss: 1.0758
Epoch   0 Batch  184/269 - Train Accuracy: 0.3941, Validation Accuracy: 0.4464, Loss: 1.2798
Epoch   0 Batch  185/269 - Train Accuracy: 0.4297, Validation Accuracy: 0.4508, Loss: 1.2324
Epoch   0 Batch  186/269 - Train Accuracy: 0.3886, Validation Accuracy: 0.4466, Loss: 1.2656
Epoch   0 Batch  187/269 - Train Accuracy: 0.4290, Validation Accuracy: 0.4525, Loss: 1.2020
Epoch   0 Batch  188/269 - Train Accuracy: 0.4403, Validation Accuracy: 0.4528, Loss: 1.1899
Epoch   0 Batch  189/269 - Train Accuracy: 0.4249, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.4908, Validation Accuracy: 0.5319, Loss: 0.9002
Epoch   1 Batch    2/269 - Train Accuracy: 0.4834, Validation Accuracy: 0.5303, Loss: 0.8869
Epoch   1 Batch    3/269 - Train Accuracy: 0.4765, Validation Accuracy: 0.5167, Loss: 0.9034
Epoch   1 Batch    4/269 - Train Accuracy: 0.4978, Validation Accuracy: 0.5289, Loss: 0.9077
Epoch   1 Batch    5/269 - Train Accuracy: 0.4754, Validation Accuracy: 0.5296, Loss: 0.9060
Epoch   1 Batch    6/269 - Train Accuracy: 0.5122, Validation Accuracy: 0.5249, Loss: 0.8383
Epoch   1 Batch    7/269 - Train Accuracy: 0.5103, Validation Accuracy: 0.5256, Loss: 0.8552
Epoch   1 Batch    8/269 - Train Accuracy: 0.4843, Validation Accuracy: 0.5295, Loss: 0.9044
Epoch   1 Batch    9/269 - Train Accuracy: 0.4963, Validation Accuracy: 0.5190, Loss: 0.8811
Epoch   1 Batch   10/269 - Train Accuracy: 0.4926, Validation Accuracy: 0.5298, Loss: 0.8839
Epoch   1 Batch   11/269 - Train Accuracy: 0.4985, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.5188, Validation Accuracy: 0.5735, Loss: 0.7821
Epoch   1 Batch   91/269 - Train Accuracy: 0.5540, Validation Accuracy: 0.5755, Loss: 0.7117
Epoch   1 Batch   92/269 - Train Accuracy: 0.5692, Validation Accuracy: 0.5824, Loss: 0.7185
Epoch   1 Batch   93/269 - Train Accuracy: 0.5766, Validation Accuracy: 0.5791, Loss: 0.6961
Epoch   1 Batch   94/269 - Train Accuracy: 0.5633, Validation Accuracy: 0.5771, Loss: 0.7259
Epoch   1 Batch   95/269 - Train Accuracy: 0.5677, Validation Accuracy: 0.5767, Loss: 0.7212
Epoch   1 Batch   96/269 - Train Accuracy: 0.5627, Validation Accuracy: 0.5744, Loss: 0.7239
Epoch   1 Batch   97/269 - Train Accuracy: 0.5457, Validation Accuracy: 0.5662, Loss: 0.7150
Epoch   1 Batch   98/269 - Train Accuracy: 0.5732, Validation Accuracy: 0.5711, Loss: 0.7149
Epoch   1 Batch   99/269 - Train Accuracy: 0.5290, Validation Accuracy: 0.5640, Loss: 0.7430
Epoch   1 Batch  100/269 - Train Accuracy: 0.5651, Validation Accuracy

...

Epoch   8 Batch  179/269 - Train Accuracy: 0.9272, Validation Accuracy: 0.9343, Loss: 0.0517
Epoch   8 Batch  180/269 - Train Accuracy: 0.9557, Validation Accuracy: 0.9344, Loss: 0.0450
Epoch   8 Batch  181/269 - Train Accuracy: 0.9329, Validation Accuracy: 0.9238, Loss: 0.0544
Epoch   8 Batch  182/269 - Train Accuracy: 0.9422, Validation Accuracy: 0.9224, Loss: 0.0485
Epoch   8 Batch  183/269 - Train Accuracy: 0.9513, Validation Accuracy: 0.9251, Loss: 0.0375
Epoch   8 Batch  184/269 - Train Accuracy: 0.9411, Validation Accuracy: 0.9302, Loss: 0.0470
Epoch   8 Batch  185/269 - Train Accuracy: 0.9557, Validation Accuracy: 0.9304, Loss: 0.0480
Epoch   8 Batch  186/269 - Train Accuracy: 0.9287, Validation Accuracy: 0.9347, Loss: 0.0451
Epoch   8 Batch  187/269 - Train Accuracy: 0.9386, Validation Accuracy: 0.9363, Loss: 0.0444
Epoch   8 Batch  188/269 - Train Accuracy: 0.9422, Validation Accuracy: 0.9348, Loss: 0.0444
Epoch   8 Batch  189/269 - Train Accuracy: 0.9404, Validation Accuracy

Epoch   9 Batch    1/269 - Train Accuracy: 0.9491, Validation Accuracy: 0.9403, Loss: 0.0430
Epoch   9 Batch    2/269 - Train Accuracy: 0.9398, Validation Accuracy: 0.9376, Loss: 0.0466
Epoch   9 Batch    3/269 - Train Accuracy: 0.9397, Validation Accuracy: 0.9428, Loss: 0.0444
Epoch   9 Batch    4/269 - Train Accuracy: 0.9292, Validation Accuracy: 0.9361, Loss: 0.0440
Epoch   9 Batch    5/269 - Train Accuracy: 0.9444, Validation Accuracy: 0.9340, Loss: 0.0446
Epoch   9 Batch    6/269 - Train Accuracy: 0.9556, Validation Accuracy: 0.9304, Loss: 0.0422
Epoch   9 Batch    7/269 - Train Accuracy: 0.9304, Validation Accuracy: 0.9303, Loss: 0.0418
Epoch   9 Batch    8/269 - Train Accuracy: 0.9465, Validation Accuracy: 0.9349, Loss: 0.0495
Epoch   9 Batch    9/269 - Train Accuracy: 0.9384, Validation Accuracy: 0.9359, Loss: 0.0458
Epoch   9 Batch   10/269 - Train Accuracy: 0.9421, Validation Accuracy: 0.9328, Loss: 0.0406
Epoch   9 Batch   11/269 - Train Accuracy: 0.9489, Validation Accuracy

Epoch   9 Batch   90/269 - Train Accuracy: 0.9367, Validation Accuracy: 0.9360, Loss: 0.0403
Epoch   9 Batch   91/269 - Train Accuracy: 0.9460, Validation Accuracy: 0.9362, Loss: 0.0394
Epoch   9 Batch   92/269 - Train Accuracy: 0.9558, Validation Accuracy: 0.9421, Loss: 0.0361
Epoch   9 Batch   93/269 - Train Accuracy: 0.9479, Validation Accuracy: 0.9386, Loss: 0.0403
Epoch   9 Batch   94/269 - Train Accuracy: 0.9447, Validation Accuracy: 0.9369, Loss: 0.0544
Epoch   9 Batch   95/269 - Train Accuracy: 0.9408, Validation Accuracy: 0.9355, Loss: 0.0440
Epoch   9 Batch   96/269 - Train Accuracy: 0.9194, Validation Accuracy: 0.9355, Loss: 0.0479
Epoch   9 Batch   97/269 - Train Accuracy: 0.9528, Validation Accuracy: 0.9327, Loss: 0.0458
Epoch   9 Batch   98/269 - Train Accuracy: 0.9495, Validation Accuracy: 0.9348, Loss: 0.0439
Epoch   9 Batch   99/269 - Train Accuracy: 0.9421, Validation Accuracy: 0.9319, Loss: 0.0413
Epoch   9 Batch  100/269 - Train Accuracy: 0.9389, Validation Accuracy

Epoch   9 Batch  179/269 - Train Accuracy: 0.9385, Validation Accuracy: 0.9367, Loss: 0.0427
Epoch   9 Batch  180/269 - Train Accuracy: 0.9609, Validation Accuracy: 0.9406, Loss: 0.0397
Epoch   9 Batch  181/269 - Train Accuracy: 0.9448, Validation Accuracy: 0.9419, Loss: 0.0434
Epoch   9 Batch  182/269 - Train Accuracy: 0.9403, Validation Accuracy: 0.9417, Loss: 0.0390
Epoch   9 Batch  183/269 - Train Accuracy: 0.9569, Validation Accuracy: 0.9451, Loss: 0.0326
Epoch   9 Batch  184/269 - Train Accuracy: 0.9475, Validation Accuracy: 0.9469, Loss: 0.0355
Epoch   9 Batch  185/269 - Train Accuracy: 0.9607, Validation Accuracy: 0.9436, Loss: 0.0408
Epoch   9 Batch  186/269 - Train Accuracy: 0.9353, Validation Accuracy: 0.9431, Loss: 0.0384
Epoch   9 Batch  187/269 - Train Accuracy: 0.9515, Validation Accuracy: 0.9428, Loss: 0.0391
Epoch   9 Batch  188/269 - Train Accuracy: 0.9537, Validation Accuracy: 0.9457, Loss: 0.0381
Epoch   9 Batch  189/269 - Train Accuracy: 0.9544, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    UNK = vocab_to_int['<UNK>']
    word_ids = [vocab_to_int[word] if word in vocab_to_int.keys() else UNK
                for word in sentence.lower().split()]

    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [21]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [193, 12, 125, 211, 149, 34, 128]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [156, 273, 108, 182, 75, 140, 1]
  French Words: il conduit une voiture rouge . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.